## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from weather_config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/tylerengalla/Desktop/Data_Bootcamp/Analysis_Projects/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Enid,US,36.40,-97.88,66.20,100,90,19.46,overcast clouds
1,1,Bambous Virieux,MU,-20.34,57.76,75.20,69,40,11.41,scattered clouds
2,2,Fort Nelson,CA,58.81,-122.70,15.80,85,90,2.24,overcast clouds
3,3,New Norfolk,AU,-42.78,147.06,55.99,63,99,1.01,overcast clouds
4,4,Chokurdakh,RU,70.63,147.92,-14.35,90,92,2.89,overcast clouds


In [73]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you would like for your trip?75


In [74]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Enid,US,36.40,-97.88,66.20,100,90,19.46,overcast clouds
6,6,Pringsewu,ID,-5.36,104.97,74.41,91,100,3.33,overcast clouds
8,8,Riosucio,CO,5.42,-75.70,69.80,88,75,2.24,fog
12,12,Hilo,US,19.73,-155.09,71.60,60,75,6.93,broken clouds
13,13,Mar Del Plata,AR,-38.00,-57.56,66.20,82,75,11.41,broken clouds
...,...,...,...,...,...,...,...,...,...,...
688,688,Sonamukhi,IN,23.30,87.42,72.21,54,0,2.24,clear sky
690,690,Gamboma,CG,-1.88,15.86,73.09,94,100,6.22,moderate rain
695,695,Naze,JP,28.37,129.48,68.00,64,75,16.11,broken clouds
698,698,Rocha,UY,-34.48,-54.33,71.04,57,100,9.04,overcast clouds


In [75]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [76]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean = preferred_cities_df.dropna()

In [77]:
preferred_cities_clean.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [78]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Enid,US,66.20,overcast clouds,36.40,-97.88,
6,Pringsewu,ID,74.41,overcast clouds,-5.36,104.97,
8,Riosucio,CO,69.80,fog,5.42,-75.70,
12,Hilo,US,71.60,broken clouds,19.73,-155.09,
13,Mar Del Plata,AR,66.20,broken clouds,-38.00,-57.56,
15,Bredasdorp,ZA,66.20,clear sky,-34.53,20.04,
22,Constitucion,CL,66.04,scattered clouds,-35.33,-72.42,
33,Rikitea,PF,73.63,light rain,-23.12,-134.97,
35,Avarua,CK,71.60,light rain,-21.21,-159.78,
40,Vaini,TO,69.80,few clouds,-21.20,-175.20,


In [79]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [80]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']

In [81]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Enid,US,66.20,overcast clouds,36.40,-97.88,Ramada by Wyndham Enid
6,Pringsewu,ID,74.41,overcast clouds,-5.36,104.97,Urban Style by Front One
8,Riosucio,CO,69.80,fog,5.42,-75.70,Finca Cafe La Ermita
12,Hilo,US,71.60,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
13,Mar Del Plata,AR,66.20,broken clouds,-38.00,-57.56,Gran Hotel Mar del Plata habitaciones
...,...,...,...,...,...,...,...
688,Sonamukhi,IN,72.21,clear sky,23.30,87.42,Shubham lodge
690,Gamboma,CG,73.09,moderate rain,-1.88,15.86,Residence N'Keni
695,Naze,JP,68.00,broken clouds,28.37,129.48,奄美ポートタワーホテル
698,Rocha,UY,71.04,overcast clouds,-34.48,-54.33,Beleza Pura B&B


In [82]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/tylerengalla/Desktop/Data_Bootcamp/Analysis_Projects/World_Weather_Analysis/Weather_Database/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [83]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>  
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [84]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))